<a href="https://colab.research.google.com/github/aTegart/abc-music-gen/blob/main/D3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Getting set up:

You will be connecting to your personal google drive, so make sure you have '/content/drive/My Drive/CS 486 Music Generation/' present (without the ending space, so Celine I changed your folder name). This code now reads from all song folders. Please make sure you are on the latest version of the songs, because on Nov 11 changes were made to filetypes and to remove accented characters. When uploading files to your personal google drive, make sure to have "convert uploads" unchecked in your settings.

Remember you need to authenticate when you remount, and copy the auth code from the link.





In [3]:
from google.colab import drive
#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


Check position


In [4]:
!pwd
!cd './drive/My Drive/'
!pwd

/content
/content


## Preprocess the songs

In [5]:
from os import listdir
from os.path import isfile, join
import os

#list songs
mypath = '/content/drive/My Drive/CS 486 Music Generation/' #this was previously "myPath" when it needed to be "mypath" 🤯
#/content/drive/My Drive/CS 486 Music Generation 
#/content/drive/My Drive/CS 486 Music Generation
folders = [f for f in listdir(mypath) if not isfile(join(mypath, f))]
print(folders)
allSongs = []
for folder in folders:
  #if folder == "montreal":
  for song in listdir(mypath + '/' + folder):
    print(folder+" "+song)
    f = open(os.path.join(mypath,folder,song), "r")
    text = f.read()
    allSongs.append(text)
print(len(allSongs))


['tune arch canada', 'canad', 'montreal']
tune arch canada Iles de la Madeleine - K A.txt
tune arch canada My Lily - K A.txt
tune arch canada Valse-Clog Guilmette - K Bb.txt
tune arch canada Ookpik Waltz - K G.txt
tune arch canada The Black Velvet Waltz - K C.txt
tune arch canada Hilo marche - K F.txt
tune arch canada Duke of Gordons Birth Day - K G.txt
tune arch canada John Morrison Assynt House - K A Dorian.txt
tune arch canada Reel du courrier - K Bb.txt
tune arch canada Patins de Pauline - K G.txt
tune arch canada Journeymans Jig - K Dmaj.txt
tune arch canada Reel de Beloeil - K D.txt
tune arch canada 91st at Modder River - Bmin.txt
tune arch canada Macton Reel - K D.txt
tune arch canada Quadrille canadien - K G.txt
tune arch canada Pappy Dailys Breakdown - K Bb.txt
tune arch canada Celtic Cousins - K F.txt
tune arch canada Alex and Maureens Two-Step - K G.txt
tune arch canada Loup Garou - K A.txt
tune arch canada Reel de la Pointe Bleue - K G.txt
tune arch canada Reel du pˆcheur -

In [6]:
print(allSongs[100])

X: 1
T: Maple Leaf Jig
C: Trad
R: jig
Z: John Chambers <jc:trillian.mit.edu>
B: NEFR #9
M: 6/8
L: 1/8
K: G
A \
| "G"B2G "C"G2E | "G"D2B, B,2G | "D7"F2A AGF | "G"G2B BGA \
| "G"B2G "C"G2E | "G"D2B, B,2G | "D7"F2A AGF | "G"G3- G2 :|
K: D
|: F/G/ \
| "D"A2A A^GA | "(A7)"BA^G "D"A2d | "A7"d2c c2e | "D"e2d dFG \
| "D"A2A A^GA | "(A7)"BA^G "D"A2d | "A7"d2c cBc |1 "D"d3- d2 :|2 "D"Hd3 "D7"=c3 |]



In [7]:
#Combine songs into one long string
spacing = "\n\n"
combined_songs = spacing.join(allSongs)

#Find vocabulary for vectorization (unique characters in string)
uniq_chars = sorted(set(combined_songs))
print(len(uniq_chars))

char_num_map = {c:i for i, c in enumerate(uniq_chars)}
num_char_map = {i:c for i, c in enumerate(uniq_chars)}

#proof that the mapping holds both ways
print('{')
for char,_ in zip(char_num_map, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char_num_map[char]))
print('  ...\n}')

print('{')
for char,_ in zip(num_char_map, range(20)):
    print('  {:4s}: {:3s},'.format(repr(char), num_char_map[char]))
print('  ...\n}')

112
{
  '\t':   0,
  '\n':   1,
  '\x1a':   2,
  ' ' :   3,
  '!' :   4,
  '"' :   5,
  '#' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  '+' :  13,
  ',' :  14,
  '-' :  15,
  '.' :  16,
  '/' :  17,
  '0' :  18,
  '1' :  19,
  ...
}
{
  0   : 	  ,
  1   : 
  ,
  2   :   ,
  3   :    ,
  4   : !  ,
  5   : "  ,
  6   : #  ,
  7   : %  ,
  8   : &  ,
  9   : '  ,
  10  : (  ,
  11  : )  ,
  12  : *  ,
  13  : +  ,
  14  : ,  ,
  15  : -  ,
  16  : .  ,
  17  : /  ,
  18  : 0  ,
  19  : 1  ,
  ...
}


In [8]:
import numpy as np
#Vectorize combined_songs
# array of a num for each char in the combined_songs string
vsongs = np.array([char_num_map[c] for c in combined_songs])

print ('{} ---- characters mapped to int ----> {}'.format(repr(combined_songs[:10]), vsongs[:10]))
# check that vectorized_songs is a numpy array
assert isinstance(vsongs, np.ndarray), "returned result should be a numpy array"

'X:414\n%\nT:' ---- characters mapped to int ----> [58 28 22 19 22  1  7  1 54 28]


In [9]:
def batch(vsongs, seqlen, bsize):
  highest_start_index = vsongs.shape[0] - 1 - seqlen
  start_indices = np.random.choice(highest_start_index, bsize)

  input = [vsongs[i : i+seqlen] for i in start_indices]
  output = [vsongs[i+1 : i+1+seqlen] for i in start_indices]

  input = np.reshape(input, [bsize, seqlen])
  output = np.reshape(output, [bsize, seqlen])

  return input, output

testin, testout = batch(vsongs, 5, 2)
#observe how testout is shifted one num forward from testin
print(testin)
print(testout)

[[28 37 93  1 46]
 [ 3 36 41 41 41]]
[[37 93  1 46 28]
 [36 41 41 41  3]]


In [10]:
#more testing, remove later

x_batch, y_batch = batch(vsongs, seqlen=5, bsize=1)

for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(num_char_map[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(num_char_map[target_idx])))

Step   0
  input: 68 ('c')
  expected output: 35 ('A')
Step   1
  input: 35 ('A')
  expected output: 3 (' ')
Step   2
  input: 3 (' ')
  expected output: 39 ('E')
Step   3
  input: 39 ('E')
  expected output: 35 ('A')
Step   4
  input: 35 ('A')
  expected output: 68 ('c')


## Setup Tensorflow and our ML env

In [11]:
%tensorflow_version 2.x
import tensorflow as tf 
from tensorflow import keras

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0

In [12]:
## Set up the RNN model
def LSTM(rnn_units): 
  return keras.layers.LSTM(
    rnn_units, 
    return_sequences=True, 
    recurrent_initializer='glorot_uniform',
    recurrent_activation='sigmoid',
    stateful=True,
  )

## Build the RNN Model
def build_RNN_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = keras.Sequential([
    # 1: Embedding layer to transform indices into dense vectors of a fixed embedding size
    keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    # 2: LSTM with rnn_units number of units
    LSTM(rnn_units),
    # 3: Dense layer that transform the LSTM output into the vocab_size
    keras.layers.Dense(vocab_size)
  ])
  return model

# Build model with default hyperparameters which can be modified
model = build_RNN_model(len(uniq_chars), embedding_dim=256, rnn_units=1024, batch_size=32)

### Test the RNN Model

In [13]:
## Prints out a summary of the model so we can check for its layers, shapes, batch sizes, etc.
model.summary()

## Check output dimemsionality
x, y = batch(vsongs, seqlen=50, bsize=32)
pred = model(x)
print("Input shape: ", x.shape)
print("Prediction shape: ", pred.shape)

## Check the prediction of the untrained RNN model using categorical distribution
random_indices = tf.random.categorical(pred[0], num_samples=1)
random_indices = tf.squeeze(random_indices, axis=-1).numpy()
random_indices

## decode the prediction from the unntrained RNN model
## We should expect a prediction that does not make much sense as it is untrained
index2char = np.array(uniq_chars)
print("Input: \n", repr("".join(index2char[x[0]])))
print("Predicted next char: \n", repr("".join(index2char[random_indices])))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 256)           28672     
_________________________________________________________________
lstm (LSTM)                  (32, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (32, None, 112)           114800    
Total params: 5,390,448
Trainable params: 5,390,448
Non-trainable params: 0
_________________________________________________________________
Input shape:  (32, 50)
Prediction shape:  (32, 50, 112)
Input: 
 '"Dm"cdc cAF | "Gm"GAG GAc |\\\n[1 "Bb"d3 "C"f3 :|[2 '
Predicted next char: 
 'pk\né^KAç5ê(FEzS5#: JY;É5h}&\x1a&’RêC“7xj{[ZêzED_çWv~<'


### Training the RNN model

In [14]:
## Train with sparse_categorical_corssentropy loss which uses integer targets for categorical classificiation tasks
## logits: predictions

def get_loss(labels, logits):
  return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

sample_loss = get_loss(y, pred) # from the sample prediction in the above section
print("Scalar loss: ", sample_loss.numpy().mean())


Scalar loss:  4.71924


### Hyperparameters

In [15]:
### Define hyperarameters
# Model params
rnn_units = 1024 # suggeted range: 1 - 2048
uniq_chars_len = len(uniq_chars)
embedding_dim = 256

# Optimization params
training_cycles = 1500 # increase to make longer trainings
batch_size = 4 # suggested range: 1 - 64
seq_len = 50 # suggested range: 50 - 500
learning_rate = 1e-4 # suggested range: 1e-5 - 1e-1

# Checkpoint location
ckpt_dir = './training_checkpoints'
ckpt_prefix = os.path.join(ckpt_dir, "ckpt")

### Train the RNN model

In [20]:
from tqdm import tqdm # progress bar

rnn_model = build_RNN_model(uniq_chars_len, embedding_dim, rnn_units, batch_size)
optimizer = keras.optimizers.Adam(learning_rate)

@tf.function
def step(x, y):
  with tf.GradientTape() as tape:
    y_hat = rnn_model(x)
    loss = get_loss(y, y_hat)
    gradients = tape.gradient(loss, rnn_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, rnn_model.trainable_variables))
    return loss

# clear if there is progress bar instance
if hasattr(tqdm, '_instances') : tqdm._instances.clear()

## Start traning
history = []

for i in tqdm(range(training_cycles)):
  x_batch, y_batch = batch(vsongs, seq_len, batch_size)
  loss = step(x_batch, y_batch)
  history.append(loss.numpy().mean())
  # Update model with updated weights
  if i%100 == 0:
    rnn_model.save_weights(ckpt_prefix)

# save model
rnn_model.save_weights(ckpt_prefix)


100%|██████████| 1500/1500 [00:22<00:00, 67.54it/s]
